<a href="https://colab.research.google.com/github/maria-becerra/LEARN_GIT/blob/master/mb_Coca_Cola_Classification_model_3_PACK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas              as pd
import numpy               as np
import re                        # Import the regular expression module

In [ ]:
# Create requirements file
#!pip freeze > requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/Capstone/Dataset.csv'

df = pd.read_csv(file_path, sep=',', low_memory=False)
df.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,STORE_CHAIN,UPC,DESCRIPTION_1,DESCRIPTION_2,DIST,MANUFACTURER,PRODUCT,SIZE_ML,COUNT,SALES,...,TARGET_SIZE_ML,TARGET_COUNT,TARGET_PACK_CONTENT,TARGET_PACK_TYPE,TARGET_PACK_SUBTYPE,TARGET_CATEGORY,TARGET_TYPE,TARGET_SWEETNER,TARGET_FLAVOUR,TARGET_FLAVOUR_GROUPED
0,LCL,0,COCA COLA CLASSIC SLIM CAN,NaN,NaN,-,Coca-Cola,2,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SDM,0,UNDEFINED,UNDEFINED,NaN,PEPSI-COLA BEV(CTL),UNKNOWN,1,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,METRO,2,pal.UncleB.rice asst. 540x1un,NaN,NaN,MARS CANADA INC,NaN,NaN,NaN,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CIRCLE K,7,LIPTON T.G.CITRON 500ML,OTHER BEVERAGES,NaN,PEPSI-COLA,NaN,1,UN,8,...,1000.0,1.0,BOTTLE - PET,FC,FUTURE CONSUMPTION,WATER,CLASSIC,ORIGINAL,ALKALINE,WATER (UNFLAVORED)
4,CIRCLE K,18,FROSTER SWIRL 350ML,Frz Disp Bevs (83),NaN,Unknown,NaN,0,Unknown,"3,382,989",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Split the dataset

The dataset was provided contains a high number of target columns that are empty. We will remove empty columns and only labeled data will remain.

In [ ]:
# Imports
from sklearn.model_selection import train_test_split

In [ ]:
# Get the column names starting with 'TARGET_'
target_columns = [col for col in df.columns if col.startswith('TARGET_')]

In [ ]:
# Drop rows with null values in target columns
df_labeled = df.dropna(subset=target_columns)

In [ ]:
df_labeled.shape

(12158, 26)

In [ ]:
# To be able to tokenize the data we will convert all columns to string type
df_labeled = df_labeled.astype(str)

In [ ]:
# Split the DataFrame into features (X) and target (y) data
X = df_labeled.drop(target_columns, axis=1)
y = df_labeled[target_columns]

# Perform stratified train-test split on the temporary DataFrame
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import sklearn
print(sklearn.__version__)


1.2.2


In [ ]:
# Print the sizes of the train, test, and validation sets
print('Train size:', len(X_train))
print('Test size:', len(X_test))

Train size: 9726
Test size: 2432


In [ ]:
X_train.head()

,STORE_CHAIN,UPC,DESCRIPTION_1,DESCRIPTION_2,DIST,MANUFACTURER,PRODUCT,SIZE_ML,COUNT,SALES,UNITS,PRICE,CATEGORY
24020,SOBEYS,6900014736,MOUNTDEW SUPERNOVA 591 ML - 6900014736,IC - OTHER FLAVOURED SODA - REGULAR,nan,PEPSICO,nan,591,nan,23,10,2.33,30-14-07 - CARBONATED SOFT DRINKS
38082,SDM,67000004971,POWERADE MIXED BERRY 591M 6PK (067000004971),POWERADE DRINK,nan,nan,nan,6,nan,"1,554,196","290,116",5.36,OTHER BEVERAGES
19409,OVERWAITEA,6700010124,Fruitopia The Grapebeyond Gb 473 Ml,Flvr Soft Drink Can,nan,Acquisition Supplier,Acquisition Brand,473ml,nan,3,2,1.50,Grocery
17949,FED CO-OP,6700000392,COCA-COLA HOLIDAY ORB BTL 400ML,SOFT DRINKS-COCA COLA 400ML,nan,nan,nan,400ML,nan,"17,031","23,677",0.72,SOFT DRINKS-COCA-COLA
19062,METRO,6700000946,Powerade blue ras.0sugar sp.drk 710ml,nan,nan,COMPAGNIE D`EMBOUTEILLAGE COCA-COLA,nan,nan,nan,"318,095","256,902",1.24,Isotonic juices and drinks


## 2. Build the model

### 2.1 Clean dataset

Write a function that cleans the dataset by removing non relevant features, removing non relevant characters.

In [ ]:
# Custom function to replace values with 6 or more digits in the 'text_column'
def replace_digits(text):
    # Use regular expression to find all sequences of digits (\d{6,})
    # and replace them with a custom string, e.g., ''
    new_text = re.sub(r'\(\d{7,}\)', '', str(text))
    new_text = re.sub(r'\d{7,}', '', new_text)
    return new_text

In [ ]:
# Function to replace all non-letter characters with an empty string
def replace_non_letters(text):
    # Replace all non-letter characters with an empty string
    cleaned_text = text.str.replace(r'[^a-zA-Z\s]', '', regex=True)

    # Handle empty or whitespace values and replace with 'UNKNOWN'
    cleaned_text = cleaned_text.apply(lambda x: 'UNKNOWN' if x.strip() == '' else x.strip())

    return cleaned_text

In [ ]:
def rewrite_presentations(text):
  # Get the matches for measure units
  matches = re.findall(r'\d+\.?\d+\s*M*L',text)
  if len(matches) > 0:
    for m in matches:
      new_measure = re.sub(r'\.0+','',m) # Remove leading zeroes for example 540.000 ML turs into 540 ML
      new_measure = re.sub(r'(\d)([A-Za-z])', r'\1 \2', new_measure)
      new_measure = re.sub(r'\s','',new_measure)
      text = text.replace(str(m), new_measure)
      matches_pack = re.findall(r'\d+\s*X\s*\d*\.*\d+\s*M*L',text)
      if len(matches_pack) > 0:
        for mp in matches_pack:
          pack_number = str(mp).split('X')[0]
          text = re.sub(r''+pack_number+'\s*X',''+pack_number+'PK ',text)
    return text
  return text

In [ ]:
def replace_dot_separated_words(text):
  return re.sub(r'(?<=[A-Za-z])\.(?=[A-Za-z])',' ', text)

In [ ]:
def remove_repeated_words(text):
    # Use regular expression to find consecutive repeated words
    pattern = r'\b(\w+)\s+\1\b'

    # Replace consecutive repeated words with a single occurrence
    cleaned_text = re.sub(pattern, r'\1', text, flags=re.IGNORECASE)

    return cleaned_text

Use word segementation for handle words that are altogether.

In [ ]:
def preprocess_dataframe(df):
    df.drop(['STORE_CHAIN', 'UPC', 'DIST',
        'MANUFACTURER', 'PRODUCT', 'SIZE_ML', 'COUNT', ' SALES ', ' UNITS ',
        ' PRICE '], axis=1, inplace=True)
    # Convert text to upper for columns that contains text
    columns_to_convert = ['DESCRIPTION_1', 'DESCRIPTION_2', 'CATEGORY']
    df[columns_to_convert] = df[columns_to_convert].apply(lambda x: x.str.upper())

    # Remove UPC that is present in some descriptions.
    df['DESCRIPTION_1'] = df['DESCRIPTION_1'].apply(replace_digits)
    df['DESCRIPTION_1'] = df['DESCRIPTION_1'].apply(rewrite_presentations)
    df['DESCRIPTION_1'] = df['DESCRIPTION_1'].apply(replace_dot_separated_words)
    df['DESCRIPTION_1'] = df['DESCRIPTION_1'].apply(remove_repeated_words)

    df['DESCRIPTION_2'] = df['DESCRIPTION_2'].apply(replace_digits)
    df['DESCRIPTION_2'] = df['DESCRIPTION_2'].apply(rewrite_presentations)
    df['DESCRIPTION_2'] = df['DESCRIPTION_2'].apply(replace_dot_separated_words)
    df['DESCRIPTION_2'] = df['DESCRIPTION_2'].apply(remove_repeated_words)

    # Replace all non digits on CATEGORY column
    df['CATEGORY'] = replace_non_letters(df['CATEGORY'])

    # Replace acronyms on DESCRIPTION_1
    # Rewrite or expand the acronyms in the text
    replacement_dict = {
        'PPK12':'12PK',
        '12PK(1)':'12PK',
        'PPK8':'8PK',
        'PPK6':'6PK',
        'PPK4':'4PK',
        'AQFNA':'AQUAFINA',
        'SPLSH':'SPLASH',
        'SPRKLG':'SPARKLING',
        'FLVR':'FLAVOUR',
        'FLV':'FLAVOUR',
        '-':' ',
        '(X)':'',
        '/': ' ',

    }

    # List of columns in which you want to perform the replacement
    columns_to_replace_packaging = ['DESCRIPTION_1', 'DESCRIPTION_2']

    # Use the replace() method on the specified columns to perform the replacement
    df[columns_to_replace_packaging] = df[columns_to_replace_packaging].replace(replacement_dict)
    df['DESCRIPTION_1'] = df['DESCRIPTION_1'].str.replace('-',' ')
    df['DESCRIPTION_2'] = df['DESCRIPTION_2'].str.replace('-',' ')

    return df


In [ ]:
X_train_new = preprocess_dataframe(X_train)

In [ ]:
X_test_new = preprocess_dataframe(X_test)

### 2.2 Tokenize text data

Since columns represent different types of textual information (e.g., one column contains product description, another contains product presentations), using separate tokenizers for each column may be beneficial. Separate tokenizers can capture different vocabularies and tailor the tokenization process to the specific semantics of each column.

In [ ]:
# Imports
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
# Extract the columns that start with "TARGET_"
target_columns = [col for col in df.columns if col.startswith('TARGET_')]

In [ ]:
# Extract the input columns (all columns except those that start with "TARGET_")
input_columns = X_train_new.columns

#### 2.2.1 Fit to train data

In [ ]:
# Tokenize the input text columns
max_words = 5000  # Choose the maximum number of words to be used in the vocabulary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train_new.values.flatten())

#### 2.2.2 Save the tokenizer for future usage
To save a Keras tokenizer for future usage, you can use the pickle module in Python, which allows you to serialize Python objects, including the Keras tokenizer, to a file.

In [ ]:
# imports
import pickle

In [ ]:
# Save the tokenizer using pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

The code below shots how to load and use the tokenizer.

In [ ]:
# Later, to load the tokenizer
#with open('tokenizer.pickle', 'rb') as handle:
#    loaded_tokenizer = pickle.load(handle)

In [ ]:
# Use the loaded tokenizer to tokenize new_texts
#sequences = loaded_tokenizer.texts_to_sequences(new_texts)
#print(sequences)

#### 2.2.3 Tokenize training data

In [ ]:
X_train_encoded = X_train_new.copy()

In [ ]:
for col in input_columns:
  X_train_encoded[col] = tokenizer.texts_to_sequences(X_train_encoded[col])
X_train_encoded.head(5)

,DESCRIPTION_1,DESCRIPTION_2,CATEGORY
24020,"[374, 1054, 26]","[51, 25, 91, 18, 30]","[4, 2, 1]"
38082,"[32, 151, 83, 1055, 43]","[32, 8]","[25, 11]"
19409,"[165, 754, 1806, 1807, 34]","[390, 2, 8, 22]",[20]
17949,"[28, 15, 708, 1808, 88, 588]","[2, 1, 28, 15, 588]","[2, 73]"
19062,"[32, 184, 709, 1809, 188, 70, 39]",[3],"[181, 19, 14, 1]"


In [ ]:
X_train_encoded = np.concatenate((
                pad_sequences(X_train_encoded['DESCRIPTION_1'],maxlen=10),
                pad_sequences(X_train_encoded['DESCRIPTION_2'],maxlen=10),
                pad_sequences(X_train_encoded['CATEGORY'],maxlen=6)
                ), axis=1)

In [ ]:
X_train_encoded[2]

array([   0,    0,    0,    0,    0,  165,  754, 1806, 1807,   34,    0,
          0,    0,    0,    0,    0,  390,    2,    8,   22,    0,    0,
          0,    0,    0,   20], dtype=int32)

#### 2.2.4 Tokenize testing data

In [ ]:
X_test_encoded = X_test_new.copy()

In [ ]:
for col in input_columns:
  X_test_encoded[col] = tokenizer.texts_to_sequences(X_test_encoded[col])
X_test_encoded.head(5)

,DESCRIPTION_1,DESCRIPTION_2,CATEGORY
25578,"[500, 702, 807]","[500, 702, 1129]","[7, 247]"
17844,"[165, 143, 347, 942, 34]",[3],"[249, 275]"
25352,"[17, 537]",[3],"[7, 534]"
23578,"[120, 45, 16, 385, 421, 26]","[79, 45, 16, 10]","[23, 11]"
38052,"[32, 29, 48, 26]","[32, 8]","[25, 11]"


In [ ]:
X_test_encoded = np.concatenate((
                pad_sequences(X_test_encoded['DESCRIPTION_1'],maxlen=10),
                pad_sequences(X_test_encoded['DESCRIPTION_2'],maxlen=10),
                pad_sequences(X_test_encoded['CATEGORY'],maxlen=6)
                ), axis=1)

In [ ]:
X_test.iloc[0]

DESCRIPTION_1           MST MULE GIN
DESCRIPTION_2     MST MULE GINGERADE
CATEGORY         ENERGY DRINKSINGLES
Name: 25578, dtype: object

In [ ]:
X_test_encoded[0]

array([   0,    0,    0,    0,    0,    0,    0,  500,  702,  807,    0,
          0,    0,    0,    0,    0,    0,  500,  702, 1129,    0,    0,
          0,    0,    7,  247], dtype=int32)

### 2.3 Encode targets

In [ ]:
target_columns

['TARGET_TPO_PRODUCT_NAME',
 'TARGET_MANUFACTURER',
 'TARGET_BRAND',
 'TARGET_SIZE_ML',
 'TARGET_COUNT',
 'TARGET_PACK_CONTENT',
 'TARGET_PACK_TYPE',
 'TARGET_PACK_SUBTYPE',
 'TARGET_CATEGORY',
 'TARGET_TYPE',
 'TARGET_SWEETNER',
 'TARGET_FLAVOUR',
 'TARGET_FLAVOUR_GROUPED']

In [ ]:
y_train_PACK_TYPE = y_train['TARGET_PACK_TYPE']
y_train_PACK_SUBTYPE = y_train['TARGET_PACK_SUBTYPE']
y_train_PACK_CONTENT = y_train['TARGET_PACK_CONTENT']

y_test_PACK_TYPE = y_test['TARGET_PACK_TYPE']
y_test_PACK_SUBTYPE = y_test['TARGET_PACK_SUBTYPE']
y_test_PACK_CONTENT = y_test['TARGET_PACK_CONTENT']

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder_PACK_TYPE = LabelEncoder()
label_encoder_PACK_SUBTYPE = LabelEncoder()
label_encoder_PACK_CONTENT = LabelEncoder()

In [ ]:
y_train_PACK_TYPE

24020    IC
38082    FC
19409    IC
17949    IC
19062    IC
         ..
48603    FC
18996    IC
19198    FC
5694     IC
23802    IC
Name: TARGET_PACK_TYPE, Length: 9726, dtype: object

In [ ]:
label_encoder_PACK_TYPE.fit(y_train_PACK_TYPE)
label_encoder_PACK_SUBTYPE.fit(y_train_PACK_SUBTYPE)
label_encoder_PACK_CONTENT.fit(y_train_PACK_CONTENT)

LabelEncoder()

In [ ]:
import joblib

In [ ]:
# Save the LabelEncoder using pickle
with open('label_encoder_PACK_TYPE.pkl', 'wb') as handle:
    pickle.dump(label_encoder_PACK_TYPE, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the LabelEncoder using pickle
with open('label_encoder_PACK_SUBTYPE.pkl', 'wb') as handle:
    pickle.dump(label_encoder_PACK_SUBTYPE, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the LabelEncoder using pickle
with open('label_encoder_PACK_CONTENT.pkl', 'wb') as handle:
    pickle.dump(label_encoder_PACK_CONTENT, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Later, to load the LabelEncoder
#with open('label_encoder_CATEGORY.pickle', 'rb') as handle:
#    loaded_label_encoder_CATEGORY = pickle.load(handle)

In [ ]:
# Later, to load the LabelEncoder
#with open('label_encoder_PACK_SUBTYPE.pkl', 'rb') as handle:
 #   loaded_label_encoder_PACK_SUBTYPE = pickle.load(handle)

In [ ]:
y_train_PACK_TYPE_encoded = label_encoder_PACK_TYPE.transform(y_train_PACK_TYPE)
y_train_PACK_SUBTYPE_encoded = label_encoder_PACK_SUBTYPE.transform(y_train_PACK_SUBTYPE)
y_train_PACK_CONTENT_encoded = label_encoder_PACK_CONTENT.transform(y_train_PACK_CONTENT)

In [ ]:
y_test_PACK_TYPE_encoded = label_encoder_PACK_TYPE.transform(y_test_PACK_TYPE)
y_test_PACK_SUBTYPE_encoded = label_encoder_PACK_SUBTYPE.transform(y_test_PACK_SUBTYPE)
y_test_PACK_CONTENT_encoded = label_encoder_PACK_CONTENT.transform(y_test_PACK_CONTENT)

Apply one hot encoding to categories

In [ ]:
from keras.utils import to_categorical

In [ ]:
#y_train_PACK_TYPE_encoded_one_hot = to_categorical(y_train_PACK_TYPE_encoded) # Not needed since it is a binary category
y_train_PACK_SUBTYPE_encoded_one_hot = to_categorical(y_train_PACK_SUBTYPE_encoded)

In [ ]:
y_train_PACK_CONTENT_encoded_one_hot = to_categorical(y_train_PACK_CONTENT_encoded)

In [ ]:
y_train_PACK_SUBTYPE_encoded_one_hot.shape

(9726, 4)

In [ ]:
#y_test_PACK_TYPE_encoded_one_hot = to_categorical(y_test_PACK_TYPE_encoded) # Not needed since it is a binary category
y_test_PACK_SUBTYPE_encoded_one_hot = to_categorical(y_test_PACK_SUBTYPE_encoded)

In [ ]:
y_test_PACK_CONTENT_encoded_one_hot = to_categorical(y_test_PACK_CONTENT_encoded)

In [ ]:
#y_test_PACK_TYPE_encoded_one_hot.shape

In [ ]:
y_test_PACK_SUBTYPE_encoded_one_hot.shape

(2432, 4)

### 2.4 Create model TYPE

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, Bidirectional, LSTM, Dense

In [ ]:
# Build the BiLSTM model for binary classification
def build_model_TYPE(input_shape):
    model = Sequential()
    model.add(Embedding(max_words, 100, input_length=26))
    model.add(Bidirectional(LSTM(128)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

    return model

In [ ]:
model = build_model_TYPE((26,))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 26, 100)           500000    
                                                                 
 bidirectional (Bidirectiona  (None, 256)              234496    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 734,753
Trainable params: 734,753
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train_encoded, y_train_PACK_TYPE_encoded, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
 49/244 [=====>........................] - ETA: 12s - loss: 0.6617 - accuracy: 0.5963

KeyboardInterrupt: ignored

#### 2.5.1 Evaluate model PACK_TYPE

In [ ]:
# Evaluate the model on the test data
evaluation_results = model.evaluate(X_test_encoded, y_test_PACK_TYPE_encoded)

# The evaluation results will contain the loss and any specified metrics
print("Test Loss:", evaluation_results[0])
print("Test Accuracy:", evaluation_results[1])

76/76 [==============================] - 4s 57ms/step - loss: 0.3995 - accuracy: 0.8429
Test Loss: 0.39950910210609436
Test Accuracy: 0.8429276347160339


In [ ]:
y_pred_probs = model.predict(X_test_encoded)

76/76 [==============================] - 5s 34ms/step


In [ ]:
# Define the threshold for classification (0.5 is commonly used)
threshold = 0.5

# Convert probabilities to integers (0 or 1) based on the threshold
y_pred = (y_pred_probs >= threshold).astype(int)

print(y_pred)

[[1]
 [0]
 [1]
 ...
 [1]
 [0]
 [0]]


In [ ]:
# Create a DataFrame with y_test and y_pred as columns
data = {'Actual': y_test_PACK_TYPE_encoded, 'Predicted': y_pred.flatten()}
df = pd.DataFrame(data)

# Create the truth table using pd.crosstab()
truth_table = pd.crosstab(df['Actual'], df['Predicted'], rownames=['Actual'], colnames=['Predicted'])


In [ ]:
truth_table

Predicted,0,1
Actual,,
0,1125,195
1,187,925


#### 2.5.2 Save the model PACK_TYPE

In [ ]:
from keras.models import save_model

# Save the model to a file named 'your_model_filename.h5'
model.save('model_PACK_TYPE.h5')

### 2.5 Create model PACK_SUBTYPE


In [ ]:
len(tokenizer.word_counts)

2939

In [ ]:
def build_model(input_shape, num_classes):
    text_input = Input(shape=input_shape, name='text_input')
    embedding_layer = Embedding(max_words, 100, input_length=26)(text_input)

    lstm_layer = Bidirectional(LSTM(128))(embedding_layer)

    output_layer = Dense(num_classes, activation='softmax')(lstm_layer)

    model = Model(inputs=text_input, outputs=output_layer)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
model = build_model((26,), len(np.unique(y_train_PACK_SUBTYPE)))

In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 26)]              0         
                                                                 
 embedding_4 (Embedding)     (None, 26, 100)           500000    
                                                                 
 bidirectional_4 (Bidirectio  (None, 256)              234496    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 735,524
Trainable params: 735,524
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train_encoded, y_train_PACK_SUBTYPE_encoded_one_hot, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
244/244 [==============================] - 40s 141ms/step - loss: 0.8425 - accuracy: 0.6238 - val_loss: 0.5945 - val_accuracy: 0.7646
Epoch 2/10
244/244 [==============================] - 36s 146ms/step - loss: 0.5087 - accuracy: 0.7990 - val_loss: 0.5258 - val_accuracy: 0.7801
Epoch 3/10
244/244 [==============================] - 35s 145ms/step - loss: 0.4079 - accuracy: 0.8379 - val_loss: 0.4591 - val_accuracy: 0.8068
Epoch 4/10
244/244 [==============================] - 35s 143ms/step - loss: 0.3717 - accuracy: 0.8476 - val_loss: 0.5213 - val_accuracy: 0.8032
Epoch 5/10
244/244 [==============================] - 38s 157ms/step - loss: 0.3515 - accuracy: 0.8531 - val_loss: 0.4531 - val_accuracy: 0.8155
Epoch 6/10
244/244 [==============================] - 35s 144ms/step - loss: 0.3238 - accuracy: 0.8677 - val_loss: 0.4464 - val_accuracy: 0.8155
Epoch 7/10
244/244 [==============================] - 35s 144ms/step - loss: 0.3121 - accuracy: 0.8726 - val_loss: 0.4598 - val_ac

#### 2.5.1 Evaluate model PACK_SUBTYPE

In [ ]:
# Evaluate the model on the test data
evaluation_results = model.evaluate(X_test_encoded, y_test_PACK_SUBTYPE_encoded_one_hot)

# The evaluation results will contain the loss and any specified metrics
print("Test Loss:", evaluation_results[0])
print("Test Accuracy:", evaluation_results[1])

76/76 [==============================] - 1s 18ms/step - loss: 0.5277 - accuracy: 0.8014
Test Loss: 0.5277420282363892
Test Accuracy: 0.8013980388641357


#### 2.5.2 Save the model PACK_SUBTYPE

In [ ]:
from keras.models import save_model

# Save the model to a file named 'your_model_filename.h5'
model.save('model_PACK_SUBTYPE.h5')

#### 2.5.3 Load the model PACK_SUBTYPE

In [ ]:
# Load the saved weights into the model
#loaded_model.load_weights('model_PACK_SUBTYPE.h5')

## 2.6 Create model TARGET_PACK_CONTENT

In [ ]:
def build_model_content(input_shape, num_classes):
    text_input = Input(shape=input_shape, name='text_input')
    embedding_layer = Embedding(max_words, 100, input_length=26)(text_input)

    lstm_layer = Bidirectional(LSTM(128))(embedding_layer)

    output_layer = Dense(num_classes, activation='softmax')(lstm_layer)

    model = Model(inputs=text_input, outputs=output_layer)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
model = build_model_content((26,), len(np.unique(y_train_PACK_CONTENT)))

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 26)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 26, 100)           500000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              234496    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 9)                 2313      
                                                                 
Total params: 736,809
Trainable params: 736,809
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train_encoded, y_train_PACK_CONTENT_encoded_one_hot, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
244/244 [==============================] - 20s 71ms/step - loss: 0.7470 - accuracy: 0.7235 - val_loss: 0.4060 - val_accuracy: 0.8664
Epoch 2/10
244/244 [==============================] - 17s 69ms/step - loss: 0.2956 - accuracy: 0.8963 - val_loss: 0.2966 - val_accuracy: 0.9003
Epoch 3/10
244/244 [==============================] - 17s 71ms/step - loss: 0.1895 - accuracy: 0.9320 - val_loss: 0.2204 - val_accuracy: 0.9203
Epoch 4/10
244/244 [==============================] - 16s 67ms/step - loss: 0.1501 - accuracy: 0.9429 - val_loss: 0.2838 - val_accuracy: 0.9065
Epoch 5/10
244/244 [==============================] - 16s 67ms/step - loss: 0.1317 - accuracy: 0.9497 - val_loss: 0.2041 - val_accuracy: 0.9322
Epoch 6/10
244/244 [==============================] - 18s 73ms/step - loss: 0.1181 - accuracy: 0.9527 - val_loss: 0.2118 - val_accuracy: 0.9270
Epoch 7/10
244/244 [==============================] - 16s 67ms/step - loss: 0.1097 - accuracy: 0.9562 - val_loss: 0.2082 - val_accuracy:

#### 2.6.1 Evaluate model PACK_CONTENT

In [ ]:
# Evaluate the model on the test data
evaluation_results = model.evaluate(X_test_encoded, y_test_PACK_CONTENT_encoded_one_hot)

# The evaluation results will contain the loss and any specified metrics
print("Test Loss:", evaluation_results[0])
print("Test Accuracy:", evaluation_results[1])

76/76 [==============================] - 1s 16ms/step - loss: 0.2202 - accuracy: 0.9309
Test Loss: 0.22019308805465698
Test Accuracy: 0.9309210777282715


#### 2.5.2 Save the model PACK_CONTENT

In [ ]:
from keras.models import save_model

# Save the model to a file named 'your_model_filename.h5'
model.save('model_PACK_CONTENT.h5')

#### 2.5.3 Load the model PACK_SUBTYPE

In [ ]:
# Load the saved weights into the model
#loaded_model.load_weights('model_PACK_SUBTYPE.h5')

In [ ]:
model.predict(X_test_encoded)

76/76 [==============================] - 3s 26ms/step


array([[2.0159762e-06, 7.8868499e-05, 9.9991459e-01, ..., 2.5499762e-06,
        6.7944620e-07, 1.9236319e-08],
       [6.2496442e-02, 1.9038582e-01, 7.3027015e-01, ..., 2.7897250e-04,
        1.3484612e-02, 2.5247328e-04],
       [2.0526254e-06, 4.5693905e-05, 9.9994737e-01, ..., 3.2789508e-06,
        6.7618981e-07, 2.3820704e-08],
       ...,
       [3.6403631e-07, 9.9878132e-01, 1.3159480e-04, ..., 1.0561356e-07,
        1.0673599e-03, 6.7270175e-06],
       [1.1946620e-06, 5.7218705e-05, 9.9993926e-01, ..., 1.4610081e-06,
        1.3342951e-07, 8.8320080e-09],
       [2.6767170e-06, 2.8660047e-04, 9.9970412e-01, ..., 3.4099198e-06,
        9.1767913e-07, 5.5795070e-08]], dtype=float32)